In [1]:
import pandas as pd
import geopandas as gpd
import json
import urllib.request
from pathlib import Path

# Build a master list of ZIP Codes

ZIP Codes change frequently, so this is challenging, and they aren't authoritatively documented in any public resource we know about. 
We'll merge together two sources, GeoNames, and a ZIP Code Business Patters (ZBP) dataset, to get the biggest list of potential ZIPs we'd need to map to a ZCTA. 

## GeoNames

The good thing about GeoNames is that each ZIP is assigned a latitude/longitude. It's not clear how those were assigned, which is a liability for this entire process, but we'll hope that they are accurate and, for ZIPs that are not ZCTAs, we'll try to locate the GeoNames coordinate in a ZCTA geometry (below).

In [2]:
gn = pd.read_csv('geonames_us_zips.csv', dtype={
    'zip': 'object',
    'county_fips': 'object'
})
gn['source'] = 'geonames'

# We know that GeoNames includes military and diplomatic ZIP Codes and ZIP Codes in the Marshall Islands, none of which have ZCTAs. 
# drop those now so we can avoid the trouble. We'll include other US Island Area postal codes, too, in case we run this with a new file.
# Puerto Rico DOES have ZCTAs
NON_ZCTA_POSTAL_ABBRS = ['AS', 'GU', 'MP', 'VI', 'FM', 'MH', 'PW', 'AA', 'AE', 'AP']
gn = gn[(gn['stusab'].notna()) & (~gn['stusab'].isin(NON_ZCTA_POSTAL_ABBRS))]
gn.head()

,country,zip,city,state,stusab,county,county_fips,community,community_code,latitude,longitude,accuracy,source
0,US,99553,Akutan,Alaska,AK,Aleutians East,013,NaN,NaN,54.1430,-165.7854,1.0,geonames
1,US,99571,Cold Bay,Alaska,AK,Aleutians East,013,NaN,NaN,55.1858,-162.7211,1.0,geonames
2,US,99583,False Pass,Alaska,AK,Aleutians East,013,NaN,NaN,54.8542,-163.4113,1.0,geonames
3,US,99612,King Cove,Alaska,AK,Aleutians East,013,NaN,NaN,55.0628,-162.3056,1.0,geonames
4,US,99661,Sand Point,Alaska,AK,Aleutians East,013,NaN,NaN,55.3192,-160.4914,1.0,geonames


## ZIP Code Business Patterns

The Census Bureau's ZIP Code Business Patterns was the original dataset we wanted to integrate with other data collected at the ZBP level. 
We'll get a bit of data from that program to give us a list of ZIP Codes that "matter".  The specific query doesn't matter much. We set the `NAICS2017` and `EMPSZES` predicates to values indicating summary statistics, so that we only get back one row per ZIP. 


In [3]:
request = urllib.request.urlopen('https://api.census.gov/data/2018/zbp?get=NAME,ZIPCODE,ESTAB&NAICS2017=00&EMPSZES=001')
data = request.read() 
raw_zbp_data = json.loads(data.decode(request.info().get_content_charset()))
zbp = pd.DataFrame(data=raw_zbp_data[1:],columns=raw_zbp_data[0])
zbp.head()

,NAME,ZIPCODE,ESTAB,NAICS2017,EMPSZES
0,"ZIP 01001 (Agawam, MA)",01001,473,00,001
1,"ZIP 01002 (Amherst, MA)",01002,539,00,001
2,"ZIP 01007 (Belchertown, MA)",01007,222,00,001
3,"ZIP 01550 (Southbridge, MA)",01550,316,00,001
4,"ZIP 01003 (Amherst, MA)",01003,20,00,001


In [4]:
# we don't need all this data
# from GeoNames, we'll use 'zip', 'city', 'stusab', 'latitude', 'longitude' -- for context, and to position ZIPs in ZCTAs
# from ZBP so we'll only merge the ZIPCODE and NAME -- for context
master_zip = gn[['zip', 'city', 'stusab', 'latitude', 'longitude', 'source']].merge(zbp[['ZIPCODE', 'NAME']],left_on='zip', right_on='ZIPCODE',how='outer')


In [5]:
master_zip = master_zip.rename(columns={
    'zip': 'geonames_zip',
    'ZIPCODE': 'zbp_zip',
    'NAME': 'zbp_title'
})
master_zip['zip_code'] = master_zip.apply(lambda x: x['geonames_zip'] if not pd.isnull(x['geonames_zip']) else x['zbp_zip'],axis=1)
master_zip['source'] = master_zip['source'].fillna('zbp')

## ZCTAs

The TIGER ZCTA shapefile provides us with a master list of ZCTAs and their geometries (boundaries). This requires `tl_2019_us_zcta510.zip`, a 500MB shapefile, which is larger than we can store in GitHub.

This code will download it if it's not available, or you can get it from https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5/

In [6]:
p = Path('tl_2019_us_zcta510.zip')
p.exists()
if not p.exists():
    print(f"{p.resolve()} not found. Downloading")
    urllib.request.urlretrieve('https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5/tl_2019_us_zcta510.zip',p.resolve())
else:
    print(f"{p} is available for use")

tl_2019_us_zcta510.zip is available for use


In [7]:
zcta_geo = gpd.read_file('zip://tl_2019_us_zcta510.zip')
zcta_geo.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327..."
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.08698 41.53780, -83.08256 41.537..."
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"MULTIPOLYGON (((-82.83558 41.71082, -82.83515 ..."
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650 41.25371, -83.48382 41.253..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.22229 41.53102, -83.22228 41.532..."


In [8]:
# update the `master_zip` data frame with all ZCTAs that match ZIP codes
mz_w_zcta = master_zip.merge(zcta_geo.rename(columns={'ZCTA5CE10': 'zcta'})[['zcta']],left_on='zip_code', right_on='zcta', how='outer')

# get rid of some of the columns we've been dragging along, and re-order
mz_w_zcta = mz_w_zcta[['zip_code', 'zcta', 'geonames_zip', 'zbp_zip', 'city', 'stusab', 'zbp_title', 'latitude', 'longitude', 'source']]
mz_w_zcta['source'] = mz_w_zcta['source'].fillna('tiger')

## Handle ZIPs with no ZCTA

How many are there?

In [9]:
print(f"ZIPs with no ZCTA: {len(mz_w_zcta[pd.isnull(mz_w_zcta['zcta'])])}")

ZIPs with no ZCTA: 7987


In [10]:
# Create a GeoDataFrame for the ZIP Codes which don't yet have ZCTAs but which do have lat/lon
temp = mz_w_zcta[pd.isnull(mz_w_zcta['zcta'])][['zip_code', 'latitude', 'longitude']].dropna() # no point in keeping null lat/lng
zip_wo_zcta_gdf = gpd.GeoDataFrame(temp,geometry=gpd.points_from_xy(temp['longitude'],temp['latitude']), 
                                   crs="EPSG:4269") # projection wasn't actually specified but this is a good bet

In [11]:
# Create a new dataframe which adds ZCTAs for ZIP Codes which can be located within some ZCTA
# only keep the useful columns from zcta_geo
geo_joined = gpd.sjoin(zip_wo_zcta_gdf,zcta_geo[['ZCTA5CE10', 'geometry']],how='inner',op='intersects')

In [12]:
geo_joined.head()

,zip_code,latitude,longitude,geometry,index_right,ZCTA5CE10
20,99509,61.2181,-149.9003,POINT (-149.90030 61.21810),19459,99501
24,99514,61.2181,-149.9003,POINT (-149.90030 61.21810),19459,99501
30,99520,61.2181,-149.9003,POINT (-149.90030 61.21810),19459,99501
31,99521,61.2181,-149.9003,POINT (-149.90030 61.21810),19459,99501
32,99522,61.2181,-149.9003,POINT (-149.90030 61.21810),19459,99501


In [13]:
# update the zcta column with values we found by geocoding
mz_w_zcta = mz_w_zcta.set_index('zip_code')
mz_w_zcta['zcta'].update(geo_joined.set_index('zip_code')['ZCTA5CE10'])

In [14]:
# what's left?
print(f"Still need {len(mz_w_zcta[pd.isnull(mz_w_zcta['zcta'])])}")

Still need 110


## Manual review

At a certain point, one runs out of technical strategies. We enlisted a student to manually review the remaining unmatched ZIP Codes. The list that student worked with was shorter than our `still_null` here, so even after including these manual updates, this process will leave ZIP Codes not in any ZCTA.  See [ZIP_ZCTA_README.md]() for more details on the method.



In [15]:
# Load in the key columns from the manual review process
manual = pd.read_csv('zcta_review.csv',
                    dtype={'zip': 'object', 'result': 'object'},
                    usecols=['zip','result']).rename(
                        columns={ 'result': 'zcta' }
                    ).dropna() # drop rows which didn't get a ZCTA
manual.head()

,zip,zcta
0,02123,02215
1,02204,02203
2,02206,02203
3,02217,02108
4,02283,02111


In [16]:
# We'll raise errors if anything in manual tries to overwrite something which 
# is not null, since the manual review was based off of a slightly different 
# starting dataset. It would probably be fine to just let them go, or to use
# overwrite=False to silently ignore manual values if mw_w_zcta already has something
mz_w_zcta.update(manual.set_index('zip'),errors='raise')

In [17]:
# what's left?
mz_w_zcta[pd.isnull(mz_w_zcta['zcta'])]

,zcta,geonames_zip,zbp_zip,city,stusab,zbp_title,latitude,longitude,source
zip_code,,,,,,,,,
96718,NaN,96718,96718,Hawaii National Park,HI,"ZIP 96718 (Hawaii National Park, HI)",19.5935,-155.4380,geonames
04737,NaN,04737,NaN,Clayton Lake,ME,NaN,46.6109,-69.5223,geonames
89023,NaN,89023,89023,Mercury,NV,"ZIP 89023 (Mercury, NV)",36.6605,-115.9945,geonames
72405,NaN,NaN,72405,NaN,NaN,"ZIP 72405 (Jonesboro, AR)",NaN,NaN,zbp
89437,NaN,NaN,89437,NaN,NaN,"ZIP 89437 (Sparks, NV)",NaN,NaN,zbp
99999,NaN,NaN,99999,NaN,NaN,ZIP 99999 (Unclassified),NaN,NaN,zbp


## This will have to do!

The three geonames addresses were ones our student reviewed and found good reasons for them not having ZCTAs.

ZIP Code 99999 isn't real, and maybe we should have just dropped it above!

[72405](https://about.usps.com/newsroom/local-releases/ar/2019/0603-new-jonesboro-zip-code.htm) and [89437](https://www.kolotv.com/content/news/Tahoe-Reno-Industrial-Center-to-get-its-own-zip-code-497853001.html) are both quite new, so may get ZCTAs in an upcoming update, or could be added to the manual review file in a future update.

In [18]:
# just keep the columns we care about
# If we were working more on this, we might somehow save the "authority" or "source" so that we would have some idea about where
# we got the ZIP Codes
temp = mz_w_zcta.reset_index()[['zip_code','zcta','source']] 

# some null zip codes got in here from the ZCTA Shapefile. Why aren't those in GeoNames or ZIP Code Business Patterns?
# Who can know? But logically, if it's a ZCTA, then we assume that it has a matching ZIP Code
temp['zip_code'] = temp['zip_code'].fillna(temp['zcta'])


In [19]:
temp.to_csv('zip_zcta_xref.csv',index=False)